In [1]:
import pymysql
import pandas as pd
import re
from textblob import TextBlob as tb
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from multiprocessing import cpu_count
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
import numpy as np
from keras.utils import np_utils
import gensim
from gensim.models.word2vec import Word2Vec 
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras import optimizers
from keras.layers import Activation
from scipy.special import softmax
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
def amazon_lable(x):
    a=-1
    if(x=='POSITIVE'):
        a = 1
    if(x=='NEGATIVE'):
        a = 0
    if(x=='NEUTRAL'):
        a = 2
    return a

def bing_lable(dum):
    a=-1
    if(dum>= 0 and dum<=0.3):
        a = 0
    if(dum>0.3 and dum<0.7):
        a = 2
    if(dum>=0.7 and dum<=1):
        a = 1
    return a    

def google_lable(x):
    a=-1
    if(x > -0.3 and x < 0.3):
        a = 2
    if( x >= 0.3 ):
        a = 1
    if(x <= -0.3):
        a = 0
    return a

def compare_apis(w,x,y):
    if(w==x==y):
        return(1)
    else:
        return(0)

def unmetric_lable(x):
    if(x=='+'):
        a = 1
    if(x=='-'):
        a = 0
    if((x==0) or (x=='0')):
        a = 2
    if(x=='U'):
        a = 3
    return a

def get_scores(x):
    xsum = sum(x)
    xneg = round(x[0]/xsum,2)
    xpos = round(x[1]/xsum,2)
    xneut = round(x[2]/xsum,2)
    return([xneg,xpos,xneut])

In [3]:
custom_stopwords = pd.read_csv("stopwords2.csv")
custom_stopwords = list(custom_stopwords['StopWords'])

def extract_words(temp):
    #temp = re.sub(r'[^\w\s]','',temp) #punctuation
    temp = re.sub(r'http\S+', '', temp) #remove links
    temp = " ".join(list(filter(lambda x:x[0]!='@', temp.split())))
    temp = re.sub(r"[^a-zA-Z0-9'\s]", ' ', temp)
    temp = temp.lower()
    temp = temp.split()
#     temp = tb(temp)        
#     lista = [word for word in temp.words if word not in stopwords.words('english')]
    lista = [word for word in temp if word not in custom_stopwords]
    listb = [lemmatizer.lemmatize(y) for y in lista]
    #listb = [stemmer.stem(y) for y in listb]
    listb = [word for word in listb if len(word)>2]
    return(listb)
        
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    i = 0
    for word in tokens:
        try:
            vec += glove_w2vec[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.  * tfidf[word]
            continue
    if count != 0:
        vec /= count
    return vec

def get_lable(b):
    a = list(b)
    i = a.index(max(a))
    if(i==0):
        lable = 0
    if(i==1):
        lable = 1
    if(i==2):
        lable = 2
    return lable

def compare(x1,x2):
    if(x1==x2):
        c = 0
    else:
        c = 1
    return c

def get_precision(x):
    df = x
    print("Precision for negatives:" +str(df.iloc[0][0]/sum(df[0])))
    print("Precision for positives:" + str(df.iloc[1][1]/sum(df[1])))
    print("Precision for neutrals:" + str(df.iloc[2][2]/sum(df[2])))

In [4]:
connection = pymysql.connect(host='ec2-75-101-133-61.compute-1.amazonaws.com',
    							 user='umtdev',
    							 password='GX^2M$4%781!ra1t-e~',
    							 db='unmetric',
    							 charset='utf8mb4',
    							 cursorclass=pymysql.cursors.DictCursor,autocommit = True)
cursor = connection.cursor()

query = str("select a.fbcomment_id,a.text,b.sentiment from fb_comment_text a join\
            fb_sentiment_comments_comprehend b on b.fbpagepostcomments_id = a.fbcomment_id;")
# query = str("select a.fbcomment_id,a.text,b.sentiment as amazon_score ,c.sentiment_score as bing_score,\
#             d.score as google_score ,e.sentiment_score as unmetric_score,e.sentiment as unmetric_sentiment from fb_comment_text a join\
#             fb_sentiment_comments_comprehend b on b.fbpagepostcomments_id = a.fbcomment_id\
#             join fb_sentiment_comments_bing c on c.fbpagepostcomments_id = a.fbcomment_id\
#             join fb_sentiment_comments_google_nlp d on d.fbpagepostcomments_id = a.fbcomment_id\
#              join fb_pagepost_comments e on e.id = a.fbcomment_id;")
request = cursor.execute(query)
comments_amazon = pd.DataFrame(cursor.fetchall())
# all_posts = pd.DataFrame(cursor.fetchall())
cursor.close()
connection.close()

In [91]:
# model_2 = model

In [5]:
comments_amazon['senti_lable'] = comments_amazon['sentiment'].apply(lambda x : amazon_lable(x))
# print(len(all_posts))
# all_posts['amazon_lable'] = all_posts['amazon_score'].apply(lambda x : amazon_lable(x))
# all_posts['bing_lable'] = all_posts['bing_score'].apply(lambda x : bing_lable(x))
# all_posts['google_lable'] = all_posts['google_score'].apply(lambda x : google_lable(x))
# all_posts['common'] = list(map(compare_apis,all_posts['amazon_lable'],all_posts['bing_lable'],all_posts['google_lable']))
# comments_amazon = all_posts[all_posts.common==1]

print(len(comments_amazon))
comments_amazon.head(2)

333955


,fbcomment_id,sentiment,text,senti_lable
0,9065432149,NEUTRAL,We would upgrade to the A380 anyway! Haha,2
1,8841832192,NEGATIVE,"""They understand this and have no problem reso...",0


In [6]:
pool = Pool(processes=cpu_count()-1) 
print("Starting Pooling...")
results =  pool.map(extract_words, comments_amazon['text'])
print("Pooling Finished...")
pool.close()
pool.join()
comments_amazon['tokens'] = results
print(comments_amazon.head())
print(len(comments_amazon))

Starting Pooling...
Pooling Finished...
   fbcomment_id sentiment                                               text  \
0    9065432149   NEUTRAL          We would upgrade to the A380 anyway! Haha   
1    8841832192  NEGATIVE  "They understand this and have no problem reso...   
2    8841832194  NEGATIVE  #topthreenotsocustomerservicemoments After bum...   
3    8841832196   NEUTRAL          Hey Laurie, can you send both please? ^EH   
4    8967296634  NEGATIVE  I am so incredibly disappointed with my servic...   

   senti_lable                                             tokens  
0            2               [would, upgrade, a380, anyway, haha]  
1            0  [understand, problem, resolving, baggage, issu...  
2            0  [topthreenotsocustomerservicemoments, bumping,...  
3            2                   [hey, laurie, can, send, please]  
4            0  [incredibly, disappointed, service, united, fa...  
333955


In [7]:
from collections import Counter
print(Counter(comments_amazon['senti_lable']))
# comments_amazon['text'] = comments_amazon['text'].apply(lambda x : x.replace('\n',''))
# comments_amazon['text'] = comments_amazon['text'].apply(lambda x : x.replace('\r',''))
# comments_amazon.to_csv("amazon_text.csv",index = False)

Counter({2: 157141, 0: 96152, 1: 79778, -1: 884})


In [8]:
comments_amazon = comments_amazon[comments_amazon.senti_lable!=-1]
print(len(comments_amazon))
print(Counter(comments_amazon['senti_lable']))

333071
Counter({2: 157141, 0: 96152, 1: 79778})


In [62]:
comments_amazon = comments_amazon.rename(columns = {'amazon_lable':'senti_lable'})
comments_amazon.head()

,fbcomment_id,sentiment,text,senti_lable,tokens
0,9065432149,NEUTRAL,We would upgrade to the A380 anyway! Haha,2,"[would, upgrade, a380, anyway, haha]"
1,8841832192,NEGATIVE,"""They understand this and have no problem reso...",0,"[understand, no, problem, resolving, baggage, ..."
2,8841832194,NEGATIVE,#topthreenotsocustomerservicemoments After bum...,0,"[topthreenotsocustomerservicemoments, bumping,..."
3,8841832196,NEUTRAL,"Hey Laurie, can you send both please? ^EH",2,"[hey, laurie, can, send, please, eh]"
4,8967296634,NEGATIVE,I am so incredibly disappointed with my servic...,0,"[incredibly, disappointed, service, united, fa..."


In [9]:
data = comments_amazon[{'text','tokens','senti_lable','fbcomment_id'}]
#,'fbcomment_id'
# data = data.sample(frac = 1)

# x_train, x_test, y_train, y_test,z_train,z_test= train_test_split(np.array(data.tokens),
#                                                             np.array(data.senti_lable),np.array(data.fbcomment_id), test_size=0.2)

# pd.DataFrame(z_train).to_csv("/external_ebs/SentimentAnalysis_FB/train_ids.csv")
# pd.DataFrame(z_test).to_csv("/external_ebs/SentimentAnalysis_FB/test_ids.csv")

# y_train = np_utils.to_categorical(y_train)
# y_test = np_utils.to_categorical(y_test)

In [10]:
data.head()

,senti_lable,tokens,text,fbcomment_id
0,2,"[would, upgrade, a380, anyway, haha]",We would upgrade to the A380 anyway! Haha,9065432149
1,0,"[understand, problem, resolving, baggage, issu...","""They understand this and have no problem reso...",8841832192
2,0,"[topthreenotsocustomerservicemoments, bumping,...",#topthreenotsocustomerservicemoments After bum...,8841832194
3,2,"[hey, laurie, can, send, please]","Hey Laurie, can you send both please? ^EH",8841832196
4,0,"[incredibly, disappointed, service, united, fa...",I am so incredibly disappointed with my servic...,8967296634


In [11]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.840B.300d.txt'
word2vec_output_file = 'glove.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
from gensim.models import KeyedVectors
glove_w2vec = KeyedVectors.load_word2vec_format('glove.word2vec', binary=False)

In [12]:
ids_train = pd.read_csv("amazon_train_ids.csv")

ids_train = np.array(ids_train['0'])

data_train = data[data.fbcomment_id.isin(ids_train)]

print(Counter(data_train['senti_lable']))

Counter({2: 125738, 0: 76878, 1: 63837})


In [117]:
data = data.sample(frac = 1)

In [13]:
# data_train = data
# data_train = data_train.sample(frac = 0.8)
x_train = np.array(data_train.tokens)

y_train = np.array(data_train.senti_lable)
y_train = np_utils.to_categorical(y_train)

# ids_test = pd.read_csv("amazon_test_ids.csv")

# ids_test = np.array(ids_test['0'])

# data_test = data[data.fbcomment_id.isin(ids_test)]

# x_test = np.array(data_test.tokens)

# y_test = np.array(data_test.senti_lable)
# y_test = np_utils.to_categorical(y_test)

google_w2vec = gensim.models.KeyedVectors.load_word2vec_format(r'/external_ebs/SentimentAnalysis_FB/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [14]:
n_dim = 300
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_train)])
#train_vecs_w2v = scale(train_vecs_w2v)

# n_dim = 300
# test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_test)])
#test_vecs_w2v = scale(test_vecs_w2v)

In [15]:
train_vecs_w2v.shape

(266453, 300)

In [16]:
model = Sequential()
model.add(Dense(32, activation='elu', input_dim=300))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))

model.add(Dense(3,activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 

#model.fit(elmo_train_scaled, y_train, epochs=15, batch_size=32, verbose=2,validation_split=0.1)

In [ ]:
model.fit(train_vecs_w2v, y_train, epochs=50, batch_size=32, verbose=2,validation_split=0.2)

Train on 213162 samples, validate on 53291 samples
Epoch 1/50
 - 38s - loss: 0.3054 - acc: 0.8740 - val_loss: 0.3220 - val_acc: 0.8599
Epoch 2/50
 - 32s - loss: 0.3041 - acc: 0.8743 - val_loss: 0.3245 - val_acc: 0.8607
Epoch 3/50
 - 31s - loss: 0.3035 - acc: 0.8746 - val_loss: 0.3272 - val_acc: 0.8593
Epoch 4/50
 - 31s - loss: 0.3040 - acc: 0.8744 - val_loss: 0.3269 - val_acc: 0.8577
Epoch 5/50
 - 31s - loss: 0.3045 - acc: 0.8742 - val_loss: 0.3247 - val_acc: 0.8593
Epoch 6/50
 - 30s - loss: 0.3033 - acc: 0.8748 - val_loss: 0.3264 - val_acc: 0.8596
Epoch 7/50


In [72]:
#to save a model
# model_glove7_amazon_json = model.to_json()

# with open("model_glove7_amazon.json", "w") as json_file:
#     json_file.write(model_glove7_amazon_json)

# model.save_weights("model_glove7_amazon.h5")
model.save("model_glove_amazon.h5")

In [18]:
from keras.models import load_model
model_glove7_amazon = load_model('model_glove7_amazon.h5')

In [24]:
#to load saved model 
json_file = open('model_glove4_amazon.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove4_amazon = model_from_json(loaded_model_json)
model_glove4_amazon.load_weights("model_glove4_amazon.h5")

In [18]:
#for training error
dum_pred = model.predict(train_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
train_prediction = list(map(get_lable,dum_pred))
#train_prediction=data_train['text'].apply(lambda x : make_predictions(x))
# train_text = data_train['text']

# for i in range(len(train_text)):
#     if(i%10000==0):
#         print(i)
#     xtext = train_text.iloc[i]
#     train_prediction.append(make_predictions(xtext))

# train_pred = train_prediction

# train_prediction = sum(train_pred,[])
y_train_lable = list(map(get_lable,y_train))
difference = list(map(compare,train_prediction,y_train_lable))

print("Training Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

266453/266453 [==============================] - 10s 37us/step
Training Accuracy : 0.8152544726462078


In [ ]:
y_actu = pd.Series(y_train_lable, name='Actual')
y_pred = pd.Series(train_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)

print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

In [66]:
#for test error
dum_pred = model.predict(test_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
test_prediction = list(map(get_lable,dum_pred))

# test_prediction=data_test['text'].progress_apply(lambda x : make_predictions(x))
y_test_lable = list(map(get_lable,y_test))
difference = list(map(compare,test_prediction,y_test_lable))
print("Test Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

66791/66791 [==============================] - 3s 38us/step
Test Accuracy : 0.8028776332140558


In [ ]:
y_actu = pd.Series(y_test_lable, name='Actual')
y_pred = pd.Series(test_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)
print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

In [45]:
#self drive cars dataset
selfdrive = pd.read_csv("selfdrive.csv")
selfdrive = selfdrive[~selfdrive.sentiment.isin(['not_relevant','2','4'])]
selfdrive['tokens'] = selfdrive['text'].apply(lambda x : extract_words(x))
print(selfdrive.head())

def selfdrive_lable(x):
    if(x=='1'):
        a = 0
    if(x=='3'):
        a = 2
    if(x=='5'):
        a = 1
    return a

y_selfdrive= np.array(selfdrive.sentiment)
y_selfdrive = list(map(selfdrive_lable,y_selfdrive))
y_selfdrive = np_utils.to_categorical(y_selfdrive)

# selfdrive_prediction = pd.read_csv("selfdrive_stanford.csv",header = None)

# selfdrive_prediction.head()

# selfdrive_prediction.columns= ["result"]

# selfdrive_prediction = list(selfdrive_prediction['result'])
x_selfdrive = np.array(selfdrive.tokens)
n_dim = 300
selfdrive_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_selfdrive)])
selfdrive_vecs_w2v = scale(selfdrive_vecs_w2v)

dum_pred = model.predict(selfdrive_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
selfdrive_prediction = list(map(get_lable,dum_pred))
y_selfdrive_lable = list(map(get_lable,y_selfdrive))
difference = list(map(compare,selfdrive_prediction,y_selfdrive_lable))

print("selfdrive Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

  sentiment  confidence_score  \
0         5            0.7579   
1         5            0.8775   
3         3            1.0000   
4         3            1.0000   
5         5            0.7187   

                                                text  \
0  Two places I'd invest all my money if I could:...   
1  Awesome! Google driverless cars will help the ...   
3  Just saw Google self-driving car on I-34. It w...   
4  Will driverless cars eventually replace taxi d...   
5  Autonomous vehicles could reduce traffic fatal...   

                                              tokens  
0  [two, place, invest, money, could, 3d, printin...  
1  [awesome, google, driverless, car, help, blind...  
3  [saw, google, self, driving, car, 34, painted,...  
4  [driverless, car, eventually, replace, taxi, d...  
5  [autonomous, vehicle, could, reduce, traffic, ...  
3727/3727 [==============================] - 0s 37us/step
selfdrive Accuracy : 0.9173598068151328


In [49]:
y_actu = pd.Series(y_selfdrive_lable, name='Actual')
y_pred = pd.Series(selfdrive_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)

print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

Precision for negatives:0.04918032786885246
Precision for positives:0.3141025641025641
Precision for neutrals:0.9528920691711389
Recall for negatives :0.05660377358490566
Recall for positives :0.4298245614035088
Recall for neutrals :0.9274521183981428


In [45]:
apple = pd.read_csv("apple.csv")
apple = apple[apple.sentiment!="not_relevant"]
apple['tokens'] = apple['text'].apply(lambda x : extract_words(x))
print(apple.head())

def apple_lable(x):
    if(x=='1'):
        a = 0
    if(x=='3'):
        a = 2
    if(x=='5'):
        a = 1
    return a

y_apple= np.array(apple.sentiment)

y_apple = list(map(apple_lable,y_apple))

y_apple = np_utils.to_categorical(y_apple)

# apple_prediction = pd.read_csv("apple_stanford.csv",header = None)

# apple_prediction.head()

# apple_prediction.columns= ["result"]

# apple_prediction = list(apple_prediction['result'])


x_apple = np.array(apple.tokens)

n_dim = 300
apple_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_apple)])
#apple_vecs_w2v = scale(apple_vecs_w2v)

dum_pred = model.predict(apple_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
apple_prediction = list(map(get_lable,dum_pred))

y_apple_lable = list(map(get_lable,y_apple))
difference = list(map(compare,apple_prediction,y_apple_lable))

print("Apple Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

  sentiment  confidence_score  \
0         3            0.8129   
1         3            1.0000   
2         5            0.8468   
3         3            0.7997   
4         1            1.0000   

                                                text  \
0  RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...   
1  My cat only chews @apple cords. Such an #Apple...   
2  Top 3 all @Apple #tablets. Damn right! http://...   
3  Apple Inc. Flash Crash: What You Need to Know ...   
4  WTF MY BATTERY WAS 31% ONE SECOND AGO AND NOW ...   

                                              tokens  
0  [rt, why, aapl, stock, mini, flash, crash, tod...  
1                       [cat, chew, cord, applesnob]  
2                    [top, all, tablet, damn, right]  
3       [apple, inc, flash, crash, need, know, aapl]  
4      [wtf, battery, 31, one, second, ago, 29, wtf]  
2359/2359 [==============================] - 0s 39us/step
Apple Accuracy : 0.7918609580330649


In [50]:
y_actu = pd.Series(y_apple_lable, name='Actual')
y_pred = pd.Series(apple_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)

print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

Precision for negatives:0.9214145383104125
Precision for positives:0.5364806866952789
Precision for neutrals:0.764378478664193
Recall for negatives :0.5637019230769231
Recall for positives :0.5952380952380952
Recall for neutrals :0.9384965831435079


In [46]:
airline = pd.read_csv("airline.csv")
airline['tokens'] = airline['text'].apply(lambda x : extract_words(x))
print(airline.head())

def airline_lable(x):
    if(x=='negative'):
        a = 0
    if(x=='neutral'):
        a = 2
    if(x=='positive'):
        a = 1
    return a

y_airline= np.array(airline.airline_sentiment)
y_airline = list(map(airline_lable,y_airline))
y_airline = np_utils.to_categorical(y_airline)

# airline_prediction = pd.read_csv("airline_stanford.csv",header = None)

# airline_prediction.head()

# airline_prediction.columns= ["result"]

# airline_prediction = list(airline_prediction['result'])
x_airline = np.array(airline.tokens)
n_dim = 300
airline_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_airline)])
#airline_vecs_w2v = scale(airline_vecs_w2v)

dum_pred = model.predict(airline_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
airline_prediction = list(map(get_lable,dum_pred))
y_airline_lable = list(map(get_lable,y_airline))
difference = list(map(compare,airline_prediction,y_airline_lable))

print("Airline Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))


  airline_sentiment  confidence_score  \
0           neutral               1.0   
1          negative               1.0   
2          negative               1.0   
3          negative               1.0   
4          positive               1.0   

                                                text  \
0                @VirginAmerica What @dhepburn said.   
1  @VirginAmerica it's really aggressive to blast...   
2  @VirginAmerica and it's a really big bad thing...   
3  @VirginAmerica seriously would pay $30 a fligh...   
4  @VirginAmerica it was amazing, and arrived an ...   

                                              tokens  
0                                             [said]  
1  [really, aggressive, blast, obnoxious, enterta...  
2                          [really, big, bad, thing]  
3  [seriously, would, pay, 30, flight, seat, didn...  
4         [amazing, arrived, hour, early, too, good]  
10768/10768 [==============================] - 0s 39us/step
Airline Accuracy : 0.64180

In [132]:
y_airline= np.array(airline.airline_sentiment)
y_airline = list(map(airline_lable,y_airline))

In [49]:
airline['actual_score'] = y_airline_lable

airline['new_score'] = airline_prediction

airline.to_csv("airline_result.csv",index = False)

In [51]:
y_actu = pd.Series(y_airline_lable, name='Actual')
y_pred = pd.Series(airline_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)

print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

Precision for negatives:0.9656593406593407
Precision for positives:0.6571287617168229
Precision for neutrals:0.2640658694373652
Recall for negatives :0.4654396186440678
Recall for positives :0.8419721871049305
Recall for neutrals :0.8243574051407588


In [48]:
products = pd.read_csv("products.csv")
products['tokens'] = products['text'].apply(lambda x : extract_words(x))
print(products.head())

def products_lable(x):
    if(x=="Negative emotion"):
        a = 0
    if(x=="I can't tell" or x == "No emotion toward brand or product"):
        a = 2
    if(x=='Positive emotion'):
        a = 1
    return a

y_products= np.array(products.sentiment)
y_products = list(map(products_lable,y_products))
y_products = np_utils.to_categorical(y_products)

# products_prediction = pd.read_csv("products_stanford.csv",header = None)

# products_prediction.columns= ["result"]

# products_prediction = list(products_prediction['result'])

x_products = np.array(products.tokens)
n_dim = 300
products_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_products)])
#products_vecs_w2v = scale(products_vecs_w2v)

dum_pred = model.predict(products_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
products_prediction = list(map(get_lable,dum_pred))
y_products_lable = list(map(get_lable,y_products))
difference = list(map(compare,products_prediction,y_products_lable))

print("Products Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))


                                                text         sentiment  \
0  .@wesley83 I have a 3G iPhone. After 3 hrs twe...  Negative emotion   
1  @jessedee Know about @fludapp ? Awesome iPad/i...  Positive emotion   
2  @swonderlin Can not wait for #iPad 2 also. The...  Positive emotion   
3  @sxsw I hope this year's festival isn't as cra...  Negative emotion   
4  @sxtxstate great stuff on Fri #SXSW: Marissa M...  Positive emotion   

                                              tokens  
0  [wesley83, 3g, iphone, hr, tweeting, rise, aus...  
1  [know, awesome, ipad, iphone, app, likely, app...  
2   [can, not, wait, ipad, also, should, sale, sxsw]  
3  [hope, year, festival, isn, crashy, year, ipho...  
4  [great, stuff, fri, sxsw, marissa, mayer, goog...  
9092/9092 [==============================] - 0s 38us/step
Products Accuracy : 0.6319841619005719


In [29]:
products['actual_score'] = y_products_lable

products['new_score'] = products_prediction

products.to_csv("products_result.csv",index = False)

In [52]:
y_actu = pd.Series(y_products_lable, name='Actual')
y_pred = pd.Series(products_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)

print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

Precision for negatives:0.42574257425742573
Precision for positives:0.659
Precision for neutrals:0.6514829182830685
Recall for negatives :0.07543859649122807
Recall for positives :0.22128945601074546
Recall for neutrals :0.939033189033189


In [19]:
combined = pd.read_csv("combined_data.csv")
combined['tokens'] = combined['text'].apply(lambda x : extract_words(x))
combined['num_words'] = combined['tokens'].apply(lambda x : len(x))
print(len(combined))
combined = combined[combined.num_words>0]
print(len(combined))

y_combined= np.array(combined.sentiment_lable)
y_combined = np_utils.to_categorical(y_combined)

x_combined = np.array(combined.tokens)
n_dim = 300
combined_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_combined)])
#combined_vecs_w2v = scale(combined_vecs_w2v)

dum_pred = model.predict(combined_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)

combined_prediction = list(map(get_lable,dum_pred))
y_combined_lable = list(map(get_lable,y_combined))
difference = list(map(compare,combined_prediction,y_combined_lable))

print("Combined Accuracy : "+str((len(difference)-sum(difference))/(len(difference))))

25946
25931
25931/25931 [==============================] - 1s 36us/step
Combined Accuracy : 0.6697774864062319


In [20]:
combined['actual_score'] = y_combined_lable
combined['new_score'] = combined_prediction
# combined.to_csv("combined_result4.csv",index = False)

In [23]:
combined['compare'] = list(map(compare_custom,combined['actual_score'],combined['new_score']))
xdel = combined[combined.compare==1]
print(len(xdel))

940


In [71]:
# model2 = model

In [70]:
y_actu = pd.Series(y_combined_lable, name='Actual')
y_pred = pd.Series(combined_prediction, name='Predicted')
df = pd.crosstab(y_actu, y_pred)
get_precision(df)

print("Recall for negatives :" + str(df.iloc[0][0]/sum(df.iloc[0])))
print("Recall for positives :" + str(df.iloc[1][1]/sum(df.iloc[1])))
print("Recall for neutrals :" + str(df.iloc[2][2]/sum(df.iloc[2])))

Precision for negatives:0.911747401451265
Precision for positives:0.6776964683423481
Precision for neutrals:0.6241732149923681
Recall for negatives :0.5162114146124805
Recall for positives :0.42634107285828665
Recall for neutrals :0.9255595607343449


In [68]:
xdum_pred = dum_pred.tolist()
print(len(xdum_pred))
xdum_pred = list(map(get_scores,xdum_pred))
print(len(xdum_pred))
xdum_pred_df = pd.DataFrame(xdum_pred2, columns=['negative','positive','neutral'])
print(xdum_pred_df.head(2))

In [22]:
def compare_custom(x,y):
    if(((x==0)and(y==1) or (x==1)and(y==0) )):
        return(1)
    else:
        return(0)

In [13]:
texti  = "Driverless cars could also end drunk driving forever and save lives.."

In [20]:
del_vecs_w2v.shape

(1, 300)

In [ ]:
del_tokens = extract_words(texti)
print(del_tokens)
del_vecs_w2v = buildWordVector(del_tokens,300)
print(del_vecs_w2v)
print(len(del_vecs_w2v))
# del_pred =model_glove4_amazon.predict(del_vecs_w2v,batch_size=1,verbose=0)
# # del_pred = 5*del_pred
# print(del_pred)
# # del_pred = preprocessing.normalize(del_pred)
# # del_pred = softmax(del_pred,axis=1)
# print(del_pred)
# # del_pred = preprocessing.scale(del_pred)
# # print(del_pred)
# print(get_scores(del_pred.tolist()[0]))

In [48]:
def make_predictions(x):
    prob_vect = [0,0,0]
    blob = tb(x)
    for sentence in blob.sentences:
        del_tokens = extract_words(str(sentence))
        del_vecs_w2v = buildWordVector(del_tokens,300)
        del_pred =model_glove2_amazon.predict(del_vecs_w2v,batch_size=1,verbose=0)
        del_pred = softmax(del_pred,axis=1)
        prob_vect += del_pred
        return(list(map(get_lable,del_pred)))

ELMO MODEL

In [ ]:
import tensorflow_hub as hub

import tensorflow as tf

data['clean_comment'] = data['tokens'].apply(lambda x : ' '.join(x))

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

list_train = [data_train[i:i+10000] for i in range(0,data_train.shape[0],10000)]

elmo_train = []

for i in range(len(list_train)):
    print(i)
    x = list_train[i]
    elmo_train.append(elmo_vectors(x['clean_comment']))

x = elmo_train[0]

len(x)

elmo_train_new = np.concatenate(elmo_train, axis = 0)

elmo_train_new[0]

elmo_train_scaled = scale(elmo_train_new)

FOR DATA PREPARATION

In [104]:
from langdetect import detect
def find_lan2(x):
    y2 = 'nolang'
    try:
        y2 = detect(x)
        if(y2=='en'):
            print("extracting...")
        else:
            return([])
    except Exception:
        return([])
        pass  
    return(y2)

xinpdata = pd.read_csv("total_data.csv")
print(len(xinpdata))

pool = Pool(processes=cpu_count()-1) 
print("Starting Pooling...")
results =  pool.map(extract_words, xinpdata['text'])
print("Pooling Finished...")
pool.close()
pool.join()
xinpdata['tokens'] = results

xinpdata['num_words'] = xinpdata['tokens'].apply(lambda x : len(x))
xinpdata = xinpdata[xinpdata.num_words > 0]

x_train = np.array(xinpdata.tokens)
n_dim = 300
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x, x_train)])

dum_pred = model_amazon.predict(train_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
train_prediction = list(map(get_lable,dum_pred))
xinpdata['amazon_result'] = train_prediction

dum_pred = model_bing.predict(train_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
train_prediction = list(map(get_lable,dum_pred))
xinpdata['bing_result'] = train_prediction

dum_pred = model_google.predict(train_vecs_w2v,batch_size=32,verbose=1)
dum_pred = softmax(dum_pred,axis=1)
train_prediction = list(map(get_lable,dum_pred))
xinpdata['google_result'] = train_prediction

xinpdata['unmetric_result'] = xinpdata['sentiment'].apply(lambda x : unmetric_lable(x))
set(xinpdata.unmetric_result)

xinpdata['common'] = list(map(compare_apis,xinpdata['amazon_result'],xinpdata['bing_result'],xinpdata['google_result'],xinpdata['unmetric_result']))
xinpdata2 = xinpdata[xinpdata.common==1]
print(len(xinpdata2))


In [ ]:
glove_w2vec['good']